In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from pandas.io.json import json_normalize
import pandas as pd
import gspread

In [2]:
## Connect to our service account
project = 
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(project, scope)
gc = gspread.authorize(credentials)

In [3]:
##Get candidate data sheet from Google Drive
spreadsheet_key = 
worksheet_name = 'tools_database'

In [4]:
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet(worksheet_name)
data = worksheet.get_all_values()
table = pd.DataFrame(data[1:], columns=data[0])

In [5]:
table.head()

,name,url,description,topic,source,software source,Sample round 1,(R&D) Is it a study?,Is it a Software?,Is it operable?,Does it have a user interface?,Rate,legal tag 1,legal tag 2,legal tag 3
0,"1,39E+08",https://github.com/andli/pymkm,Python wrapper for the cardmarket.com API. Als...,personal+pricing,Github,Software registry,,,,,,,,,
1,95827680,https://github.com/linglong97/RunescapeTrading...,A personal project which attempts to use an al...,personal+pricing,Github,Software registry,,,,,,,,,
2,95339076,https://github.com/kylebarron/flight-scraper,Personal flight scraper to notify me for good ...,personal+pricing,Github,Software registry,,,,,,,,,
3,"1,8E+08",https://github.com/lesterchan/products-stock-c...,Just a personal project to check products pric...,personal+pricing,Github,Software registry,,,,,,,,,
4,3141411,https://github.com/chenliu/Amazon-Personal-Wis...,Amazon Personal Wish List Price Tracker,personal+pricing,Github,Software registry,,,,,,,,,


In [6]:
#duplicates identification
list_id = list(table.url)
list_uniques = list(table.url.unique())
print('Total tools: {}\nUnique tools: {}\n======='.format(len(list_id),len(list_uniques)))
for i in range(0,10):
    sets = list(set([x for x in list_id if list_id.count(x) > i]))
    print('url with {} duplicates: {}'.format(i, len(sets)))
print('=======\nlines to remove: {}'.format(len(list_id) - len(list_uniques)))

Total tools: 1747
Unique tools: 1582
url with 0 duplicates: 1582
url with 1 duplicates: 127
url with 2 duplicates: 26
url with 3 duplicates: 9
url with 4 duplicates: 2
url with 5 duplicates: 1
url with 6 duplicates: 0
url with 7 duplicates: 0
url with 8 duplicates: 0
url with 9 duplicates: 0
lines to remove: 165


In [8]:
## Save the data back to a new sheet in the dataframe
wks_name = 'without_duplicates'
table = table.drop_duplicates(subset ='url')
d2g.upload(table, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'without_duplicates' id:248885610>